In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time 

In [2]:
url= 'https://www.qiyas.sa/en/Statistics-Data/Schools/Pages/DetResults.aspx'

#Create a new Safari browser object
driver = webdriver.Safari()
driver.implicitly_wait(100)

#Go to the website
driver.get(url)

##Extract data for Capabilities, Girls, Theoretical, Private##
    
#Click on 'Capabilities'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[1]/td/div/div[1]/div').click()

#Click on 'Girls'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[2]/td/div/div[2]/div/label').click() 

#Click on 'Theoretical'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[3]/td/div/div[2]/div/label').click() 

#Click on 'Private'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[4]/td/div/div[2]/div').click() 

#Click on 'View the order of all schools'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[5]/td/span[1]/label').click() 

#Click on 'Submit'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[2]/div').click()

In [3]:
#Retrieving table from first page 
time.sleep(5)
soup_cbsp= BeautifulSoup(driver.page_source, 'html.parser')

#Parse and extract data 
table= soup_cbsp.find_all(class_='table-responsive')[1]

rows=table.find_all('tr')

COLUMNS= ['General Rank', 'Ministerial Number', 'School Name', 
          'City', 'Region', 'School Type', 'Category', 'Average']

sub_df=[]
for row in rows:
    cells= row.select('td')
    cells=cells[0:8]
    sub_df.append([cell.text.strip() for cell in cells])

data_cgtp = pd.DataFrame(sub_df, columns=COLUMNS).drop([0,1], axis=0).reset_index(drop=True)
data_cgtp

,General Rank,Ministerial Number,School Name,City,Region,School Type,Category,Average
0,2,96662,ثانوية دور الذكرالأهلية لتحفيظ القرآن الكريم,الرياض,Riyadh,Domestic,تحفيظ بنات,81.73
1,3,966465,ثانوية الرواد الأهلية ( الازدهار ) - مقررات,الرياض,Riyadh,Domestic,مقررات,81.38
2,16,35066,ثانوية الذكر بجدة نظام مقررات,جدة,Makkah,Domestic,مقررات,79.57
3,20,97424,ثانوية الأرقم الأهلية لتحفيظ القرآن الكريم,الرياض,Riyadh,Domestic,تحفيظ بنات,79.27
4,21,195029,ثانوية الفرسان الأهلية - المقررات,الرياض,Riyadh,Domestic,مقررات,79.17
5,39,28252,ثانوية دار البشرى الأهلية بمكة المكرمة,مكة المكرمة,Makkah,Domestic,تعليم عام بنات,77.33
6,49,59008,ثانوية دور العلوم الأهلية,الأحساء,AlSharquia,Domestic,تعليم عام بنات,76.64
7,53,42347,مدارس العقيق الأهلية - المرحلة الثانوية (مقررات),المدينة المنورة,Madina,Domestic,مقررات,76.44
8,56,35179,ثانوية دارالقلم النموذجية الأهلية,جدة,Makkah,Domestic,تعليم عام بنات,76.39
9,57,590215,مدارس أكاديمية الكفاح الأهلية للتعليم - الثانو...,الأحساء,AlSharquia,Domestic,مقررات,76.33


In [4]:
while True:
    try:
        next_page_btn= driver.find_elements(By.CLASS_NAME, "Next")
        if not next_page_btn:
            print("No next button")
            break
        next_btn= next_page_btn[0]
        if next_btn.get_attribute("class")== "aspNetDisabled Next":
            print("No more pages left")
            break
        else:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "Next"))).click()
        
            time.sleep(10)
       
            soup_pgs= BeautifulSoup(driver.page_source, 'html.parser')
        
            #Parse and extract data 
            table_pgs= soup_pgs.find_all(class_='table-responsive')[1]
        
            rows_pgs=table_pgs.find_all('tr')
        
            COLUMNS_pgs= ['General Rank', 'Ministerial Number', 'School Name', 
              'City', 'Region', 'School Type', 'Category', 'Average']
        
            sub_df_pgs=[]
            for row_pgs in rows_pgs:
                cells_pgs= row_pgs.select('td')
                cells_pgs=cells_pgs[0:8]
                sub_df_pgs.append([cell_pgs.text.strip() for cell_pgs in cells_pgs])
        
            data_pgs = pd.DataFrame(sub_df_pgs, columns=COLUMNS_pgs).drop([0,1], axis=0).reset_index(drop=True)
            data_cgtp=data_cgtp.append(data_pgs, ignore_index=True)
    except TimeoutException:
        time.sleep(10)
        print("Exception has been thrown. No more pages left")
        break
driver.quit()

No more pages left


In [5]:
data_cgtp

,General Rank,Ministerial Number,School Name,City,Region,School Type,Category,Average
0,2,96662,ثانوية دور الذكرالأهلية لتحفيظ القرآن الكريم,الرياض,Riyadh,Domestic,تحفيظ بنات,81.73
1,3,966465,ثانوية الرواد الأهلية ( الازدهار ) - مقررات,الرياض,Riyadh,Domestic,مقررات,81.38
2,16,35066,ثانوية الذكر بجدة نظام مقررات,جدة,Makkah,Domestic,مقررات,79.57
3,20,97424,ثانوية الأرقم الأهلية لتحفيظ القرآن الكريم,الرياض,Riyadh,Domestic,تحفيظ بنات,79.27
4,21,195029,ثانوية الفرسان الأهلية - المقررات,الرياض,Riyadh,Domestic,مقررات,79.17
5,39,28252,ثانوية دار البشرى الأهلية بمكة المكرمة,مكة المكرمة,Makkah,Domestic,تعليم عام بنات,77.33
6,49,59008,ثانوية دور العلوم الأهلية,الأحساء,AlSharquia,Domestic,تعليم عام بنات,76.64
7,53,42347,مدارس العقيق الأهلية - المرحلة الثانوية (مقررات),المدينة المنورة,Madina,Domestic,مقررات,76.44
8,56,35179,ثانوية دارالقلم النموذجية الأهلية,جدة,Makkah,Domestic,تعليم عام بنات,76.39
9,57,590215,مدارس أكاديمية الكفاح الأهلية للتعليم - الثانو...,الأحساء,AlSharquia,Domestic,مقررات,76.33


In [6]:
data_cgtp.to_csv('data/cap_girls_the_priv.csv', index=False)